In [3]:
import numpy as np
import cv2

capp = ['./SquatCounting/Squat1_8_9.avi','./SquatCounting/Squat2_16_17.avi','./SquatCounting/Squat3_11_9_10.avi']
true = [[8,9],[16,17],[11,9,10]]

cap = cv2.VideoCapture(capp[2])
_,bg = cap.read()
count = np.zeros((5,1))
true_x = np.zeros((5,1))
true_y = np.zeros((5,1))
last_x = np.zeros((5,5))
last_y = np.zeros((5,5))
islast = [False,False,False]
iscount = [False,False,False]
bg_time = 0
max_people = 0
while(cap.isOpened()):
    if(bg_time < 50):
        _,bg = cap.read()
        bg_time = bg_time + 1
        continue
    haveFrame,im = cap.read()

    if (not haveFrame) or (cv2.waitKey(1) & 0xFF == ord('q')):
        break

    diffc = cv2.absdiff(im,bg)
    diffg = cv2.cvtColor(diffc,cv2.COLOR_BGR2GRAY)
    bwmask = cv2.inRange(diffg,40,170)

    bwmask = cv2.medianBlur(bwmask,19)

    kernel = np.ones((170,7), np.uint8)
    bwmask = cv2.morphologyEx(bwmask, cv2.MORPH_DILATE, kernel)

    temp = bwmask.copy()
    contours,hierarchy = cv2.findContours(temp, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    contours = sorted(contours,key=cv2.contourArea,reverse=True)

    for i,cnt in enumerate(contours[:3]):
        x, y, w, h = cv2.boundingRect(cnt)
        if(h > 150 and h > w and 150 > w > 50):
            cv2.rectangle(im, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.rectangle(bwmask, (x, y), (x + w, y + h), (255,255, 255), 2)
            distance_x = 0
            distance_y = 0
            isStay = 0
            for c in range(5):
                distance_x += abs(last_x[i][c] - x)
                distance_y += abs(last_y[i][c] - y)
            if(distance_x == 0 and distance_y == 0):
                true_x[i] = x
                true_y[i] = y
            if(distance_x == 0 and distance_y == 0 and islast[i] == False):
                true_x[i] = x
                true_y[i] = y
                islast[i] = True

            if(islast[i] and abs(true_x[i] - x) < 20):
                if(last_y[i][0] < y and iscount[i] == False and abs(true_y[i] - y) > 12):
                    count[i] = count[i] + 1
                    iscount[i] = True
                if(iscount[i] and abs(y- true_y[i]) <= 2  and abs(y - last_y[i][0]) < 5):
                    iscount[i] = False
            for c in range(4,-1,-1):
                if(c == 0):
                    last_x[i][c] = x
                    last_y[i][c] = y
                else:
                    last_x[i][c] = last_x[i][c-1]
                    last_y[i][c] = last_y[i][c-1]
            cv2.putText(im,str(int(count[i])),(x+75,100),cv2.FONT_HERSHEY_PLAIN,4, (255,0,0), 3, cv2.LINE_AA)        

    cv2.imshow('bwmask', bwmask)
    cv2.moveWindow('bwmask',10,10)
    cv2.imshow('im', im)
    cv2.moveWindow('im', 800, 10)
cap.release()
cv2.destroyAllWindows()